## 👥 Autores

Este proyecto fue desarrollado por:

| Nombre Completo                     | Número de Identificación | Carrera Universitaria              |
| ----------------------------------- | -----------------------: | ---------------------------------- |
| **Yorladys Argumedo Lozano**        | `1038824209`            | Ingeniería Industrial Virtual      |
| **Sebastian Gabriel Castro**        | `1029720632`            | Ingeniería Industrial Virtual      |

---


In [2]:
# Importamos la librería de Google Colab para manejar archivos
from google.colab import files

# Instalamos la librería de Kaggle
!pip install kaggle --quiet

print("Por favor, sube el archivo 'kaggle.json' que descargaste de Kaggle:")
# Este comando abrirá una ventana para que selecciones el archivo desde tu PC
uploaded = files.upload()

Por favor, sube el archivo 'kaggle.json' que descargaste de Kaggle:


Saving kaggle.json to kaggle.json


In [3]:
# 1. Creamos la carpeta donde Kaggle espera encontrar las credenciales
!mkdir -p ~/.kaggle

# 2. Copiamos el archivo .json a esa carpeta
!cp kaggle.json ~/.kaggle/

# 3. Le damos los permisos de seguridad correctos al archivo
!chmod 600 ~/.kaggle/kaggle.json

print("¡API de Kaggle configurada!")

# 4. AHORA SÍ: Usamos la API para descargar los datos de la competencia
# Este comando le ordena a Colab que descargue los archivos por ti
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

print("\n¡Datos de la competencia descargados!")

# 5. Descomprimimos el archivo .zip que se acaba de descargar
!unzip -q -o udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

print("¡Archivos descomprimidos y listos para usar!")

¡API de Kaggle configurada!
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 925MB/s]

¡Datos de la competencia descargados!
¡Archivos descomprimidos y listos para usar!


In [ ]:
# ==============================================================================
# FASE 2: PREPROCESADO Y GENERACIÓN DE CSVs
# (Ejecutar DESPUÉS de haber descargado los datos con la API de Kaggle)
# ==============================================================================

import pandas as pd
import numpy as np
import os
import gc
from sklearn.preprocessing import LabelEncoder

# 1. CARGA DE DATOS (Lectura simple, ya que los archivos ya existen)
print("📂 Leyendo archivos CSV...")
try:
    df_train = pd.read_csv('train.csv')
    df_test = pd.read_csv('test.csv')
    print(f"✅ Datos cargados: Train {df_train.shape} | Test {df_test.shape}")
except FileNotFoundError:
    print("❌ ERROR: No se encuentran train.csv o test.csv. Ejecuta el bloque de descarga primero.")
    # Paramos la ejecución si no hay datos
    raise

test_ids = df_test['ID']
n_train = len(df_train)

# 2. DEFINICIÓN DE LA INGENIERÍA DE CARACTERÍSTICAS
def engineer_features(df):
    print("   ⚙️ Aplicando ingeniería de características...")

    # --- Mapeos Base ---
    map_estrato = {'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6, 'Sin Estrato': 0}
    map_edu = {
        'Ninguno': 0, 'No sabe': 0, 'No Aplica': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'TÃ©cnica o tecnolÃ³gica incompleta': 5, 'Técnica o tecnológica incompleta': 5,
        'TÃ©cnica o tecnolÃ³gica completa': 6, 'Técnica o tecnológica completa': 6,
        'EducaciÃ³n profesional incompleta': 7, 'Educación profesional incompleta': 7,
        'EducaciÃ³n profesional completa': 8, 'Educación profesional completa': 8, 'Postgrado': 9
    }

    if 'F_ESTRATOVIVIENDA' in df.columns:
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(map_estrato).fillna(1).astype('int8')

    for col in ['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']:
        if col in df.columns:
            df[col] = df[col].map(map_edu).fillna(1).astype('int8')

    # Binarias
    for col in [c for c in df.columns if 'TIENE' in c or 'PAGO' in c]:
        df[col] = np.where(df[col].astype(str).str.upper().str.contains('S|1|YES|TRUE'), 1, 0).astype('int8')

    # --- Contexto (Groupby) ---
    if 'E_PRGM_DEPARTAMENTO' in df.columns and 'F_ESTRATOVIVIENDA' in df.columns:
        df['DEPARTAMENTO_ESTRATO_MEAN'] = df.groupby('E_PRGM_DEPARTAMENTO')['F_ESTRATOVIVIENDA'].transform('mean')

    if 'E_PRGM_ACADEMICO' in df.columns:
        df['PROGRAMA_FREQ'] = df.groupby('E_PRGM_ACADEMICO')['E_PRGM_ACADEMICO'].transform('count')

    # --- Limpieza Categórica ---
    cat_cols = []
    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]) and col != 'RENDIMIENTO_GLOBAL' and col != 'ID':
            df[col] = df[col].astype(str).replace('nan', 'MISSING')
            cat_cols.append(col)

    return df, cat_cols

# 3. EJECUCIÓN
print("⚡ Uniendo y procesando...")
target = df_train['RENDIMIENTO_GLOBAL'].copy()
df_train_no_target = df_train.drop(columns=['RENDIMIENTO_GLOBAL'], errors='ignore')

# Unimos para procesar todo junto
X_all = pd.concat([df_train_no_target, df_test], axis=0).reset_index(drop=True)
X_all, cat_features = engineer_features(X_all)

# Label Encoding (Para que todo sean números en el CSV)
print(f"   🔡 Codificando {len(cat_features)} columnas categóricas...")
for col in cat_features:
    le = LabelEncoder()
    X_all[col] = le.fit_transform(X_all[col].astype(str))

# 4. GUARDADO
print("✂️ Separando y guardando archivos finales...")
X_train_processed = X_all.iloc[:n_train].copy()
X_test_processed = X_all.iloc[n_train:].copy()

# Mapeamos el target a números (0, 1, 2, 3)
target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
X_train_processed['RENDIMIENTO_GLOBAL'] = target.map(target_map)

# Crear carpeta output
os.makedirs("output", exist_ok=True)
X_train_processed.to_csv("output/train_processed_contextual.csv", index=False)
X_test_processed.to_csv("output/test_processed_contextual.csv", index=False)

print("\n✅ ¡LISTO! Archivos generados en la carpeta 'output/':")
print("   1. train_processed_contextual.csv")
print("   2. test_processed_contextual.csv")

📂 Leyendo archivos CSV...
✅ Datos cargados: Train (692500, 21) | Test (296786, 20)
⚡ Uniendo y procesando...
   ⚙️ Aplicando ingeniería de características...
